<a href="https://colab.research.google.com/github/nicikess/hsg-ml-course/blob/main/notebooks/ml_challenge_CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
import os
import glob
#import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from PIL import Image

!pip install rasterio
import rasterio
from rasterio.plot import reshape_as_image

#from tqdm import tqdm
#from tqdm.notebook import tqdm
#import seaborn as sns

from datetime import datetime

from tifffile import imread

from sklearn.metrics import confusion_matrix
from sklearn import model_selection

import torch, torchvision
import torchvision.models as models
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision.transforms import transforms
from torchvision.transforms import ToTensor
#rom torchvision.utils import make_grid
#import torch.nn.functional as F

     |████████████████████████████████| 19.3 MB 366 kB/s 


In [ ]:
import os
folder = '/content/drive/MyDrive/ML/data/dataset/trainset/'
subfolders = []
for dirs in os.walk(folder):
  subfolders.append(dirs[0])

#remove directory
subfolders.pop(0)
for i in range(0, len(subfolders)):
     print(subfolders[i])

/content/drive/MyDrive/ML/data/dataset/trainset/AnnualCrop
/content/drive/MyDrive/ML/data/dataset/trainset/HerbaceousVegetation
/content/drive/MyDrive/ML/data/dataset/trainset/Forest
/content/drive/MyDrive/ML/data/dataset/trainset/Highway
/content/drive/MyDrive/ML/data/dataset/trainset/Industrial
/content/drive/MyDrive/ML/data/dataset/trainset/Pasture
/content/drive/MyDrive/ML/data/dataset/trainset/PermanentCrop
/content/drive/MyDrive/ML/data/dataset/trainset/Residential
/content/drive/MyDrive/ML/data/dataset/trainset/River
/content/drive/MyDrive/ML/data/dataset/trainset/SeaLake


In [ ]:
ROOT_PATH = '/content/drive/MyDrive/ML/'
BASE_PATH = os.path.join(ROOT_PATH, 'data')
DATA_PATH = os.path.join(ROOT_PATH, 'dataset')
MODEL_PATH = os.path.join(ROOT_PATH, 'model')

file_path = '/content/drive/MyDrive/ML/data/dataset/train.csv'

data_df = pd.read_csv(file_path)

data_df.columns = data_df.columns.str.replace('path', 'image_id')
data_df.columns = data_df.columns.str.replace('Label', 'label')

print(data_df.columns)

#data_df = data_df.iloc[0]

data_df.head()

Index(['image_id', 'label'], dtype='object')


,image_id,label
0,AnnualCrop/AnnualCrop_135.tif,AnnualCrop
1,AnnualCrop/AnnualCrop_692.tif,AnnualCrop
2,AnnualCrop/AnnualCrop_2218.tif,AnnualCrop
3,AnnualCrop/AnnualCrop_2679.tif,AnnualCrop
4,AnnualCrop/AnnualCrop_926.tif,AnnualCrop


In [ ]:
# init deterministic seed
seed_value = 1234
np.random.seed(seed_value) # set numpy seed
torch.manual_seed(seed_value) # set pytorch seed CPU

In [ ]:
IDX_CLASS_LABELS = {
    0: 'AnnualCrop',
    1: 'Forest', 
    2: 'HerbaceousVegetation',
    3: 'Highway',
    4: 'Industrial',
    5: 'Pasture',
    6: 'PermanentCrop',
    7: 'Residential',
    8: 'River',
    9: 'SeaLake'
}

CLASSES = ['AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway', 'Industrial', 'Pasture','PermanentCrop','Residential','River', 'SeaLake']
CLASS_IDX_LABELS = dict()
for key, val in IDX_CLASS_LABELS.items():
  CLASS_IDX_LABELS[val] = key

NUM_CLASSES = len(IDX_CLASS_LABELS.items())
torch.manual_seed(10)
VALID_SIZE = 0.1

In [ ]:
## Give idx of each class name
def encode_label(label):
    idx = CLASS_IDX_LABELS[label] 
    return idx

## Take in idx and return the class name
def decode_target(target, text_labels=True):
    result = []
    if text_labels:
        return IDX_CLASS_LABELS[target]
    else:
        return target

## Show batches of images
def show_batch(dl):
    for images, labels in dl:
        print(images.shape)
        fig, ax = plt.subplots(figsize=(16, 8))
        ax.set_xticks([]); ax.set_yticks([])
        ax.imshow(make_grid(images, nrow=16).permute(1, 2, 0))
        break

In [ ]:
"""import os

def normalizeBand(band):
  band_norm = np.interp(band, (band.min(), band.max()), (0, 1))
  return band_norm

data = {}

for i in range(len(subfolders)):

  txtfiles = []

  # traverse whole directory
  for root, dirs, files in os.walk(subfolders[i]):
    # select file name
    for file in files:
        # check the extension of files
        if file.endswith('.tif'):
            txtfiles.append(os.path.join(root, file))

  code = subfolders[i].split('/')[8]
  index = encode_label(code)
  arr = np.zeros([len(txtfiles), 64, 64, 13], dtype="float32")

  print(code)
  print(index)

  i = 0
  for pic in txtfiles:
    #print(i)
    with rasterio.open(pic, "r") as img:
      tmp = np.moveaxis(img.read(), 0, 2)
      arr[i] = normalizeBand(tmp)
      i += 1
      
  data[index] = arr

AnnualCrop
0
HerbaceousVegetation
2
Forest
1
Highway
3
Industrial
4
Pasture
5
PermanentCrop
6
Residential
7
River
8
SeaLake
9


In [ ]:
for i in range(len(data)):
  key = data.get(i)
  np.save('/content/drive/MyDrive/ML/data/dataset/trainset/'+f'data_class{key}.npy', data[i], allow_pickle=True, fix_imports=True)

0
1
2
3
4
5
6
7
8
9


In [48]:
import os
path = '/content/drive/MyDrive/ML/data/dataset/trainset/'
npyfiles = []
data = {}

for root, dirs, files in os.walk(path):
    # check the extension of files
    for file in files:
        # check the extension of files
        if file.endswith('.npy'):
            npyfiles.append(os.path.join(root, file))

for i in range(len(npyfiles)):
  path = npyfiles[i]
  arr = np.load(path)
  print(path)
  index = path.split('/')[8].split('_')[1][5]
  print(index)
  print(arr.shape)
  data[i] = arr

/content/drive/MyDrive/ML/data/dataset/trainset/data_class1.npy
1
(3000, 64, 64, 13)
/content/drive/MyDrive/ML/data/dataset/trainset/data_class2.npy
2
(3000, 64, 64, 13)
/content/drive/MyDrive/ML/data/dataset/trainset/data_class3.npy
3
(2500, 64, 64, 13)
/content/drive/MyDrive/ML/data/dataset/trainset/data_class4.npy
4
(2548, 64, 64, 13)
/content/drive/MyDrive/ML/data/dataset/trainset/data_class5.npy
5
(2055, 64, 64, 13)
/content/drive/MyDrive/ML/data/dataset/trainset/data_class6.npy
6
(2500, 64, 64, 13)
/content/drive/MyDrive/ML/data/dataset/trainset/data_class7.npy
7
(3000, 64, 64, 13)
/content/drive/MyDrive/ML/data/dataset/trainset/data_class8.npy
8
(2500, 64, 64, 13)
/content/drive/MyDrive/ML/data/dataset/trainset/data_class9.npy
9
(3009, 64, 64, 13)
/content/drive/MyDrive/ML/data/dataset/trainset/data_class0.npy
0
(3000, 64, 64, 13)


In [49]:
for i in range(len(data)):
  print(len(data[i].shape))

4
4
4
4
4
4
4
4
4
4


KeyError: ignored

In [ ]:
def normalize_for_display(band_data):
    """Normalize multi-spectral imagery across bands.
    The input is expected to be in HxWxC format, e.g. 64x64x13.
    To account for outliers (e.g. extremly high values due to
    reflective surfaces), we normalize with the 2- and 98-percentiles
    instead of minimum and maximum of each band.
    """
    band_data = np.array(band_data)
    lower_perc = np.percentile(band_data, 2, axis=(0, 1))
    upper_perc = np.percentile(band_data, 98, axis=(0, 1))

    return (band_data - lower_perc) / (upper_perc - lower_perc)

In [ ]:
#Before appling transform we have a different shape
#Transform reshapes the image
#Added normalize because it was taken out below (maybe other way works as well)

from pathlib import Path
class EuroSAT(Dataset):
    def __init__(self, train_df, train_dir, transform=None):
        self.train_dir = train_dir
        self.train_df = train_df
        self.transform = transform

    def __len__(self):
        return len(self.train_df)
    
    def __getitem__(self, idx):
        row = self.train_df.loc[idx]
        img_id, label = row['image_id'], row['label']
        img_id = int(img_id.split('.')[0].split('_')[1])
        label_encode = int(encode_label(label))
        print(img_id)
        print(label_encode)
        id = img_id - 1
        img = data[label_encode][id]
        img = reshape_as_image(img)
        img_nor = normalize_for_display(img)
        img_rgb = img_nor[:, :, [3,2,1]]
        if self.transform:
            img_rgb = self.transform(img_rgb)
        return img_rgb, encode_label(label)

In [ ]:
data_transform = transforms.Compose([
                                transforms.Resize(size=(224, 224)),
#                                 transforms.CenterCrop(224),
                                transforms.ToTensor(),
                                # transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                ])

transf = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),transforms.ConvertImageDtype(torch.float), torchvision.transforms.CenterCrop(32)])

train_ds = EuroSAT(data_df, BASE_PATH, transf)
print(len(train_ds))

27000


In [ ]:
#print(train_ds[0][0])
np.shape(train_ds[0][0])

135
0


torch.Size([3, 32, 32])

In [ ]:
for i in range(len(data)):
  print(len(data[i]))

3000
3000
3000
2500
2548
2055
2500
3000
2500
3009


DONT CHANGE THE SHAPE OF THE IMAGE FROM TRAINING - ONLY FOR VISUALIZING BECAUSE LIBRARY LIKE rasterio use different dimensions


Model

In [ ]:
# implement the CIFAR10Net network architecture
class CIFAR10Net(nn.Module):
    
    # define the class constructor
    def __init__(self):
        
        # call super class constructor
        super(CIFAR10Net, self).__init__()
        
        # specify convolution layer 1
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5, stride=1, padding=0)
        
        # define max-pooling layer 1
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # specify convolution layer 2
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0)
        
        # define max-pooling layer 2
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # specify fc layer 1 - in 16 * 5 * 5, out 120
        self.linear1 = nn.Linear(16 * 5 * 5, 120, bias=True) # the linearity W*x+b
        self.relu1 = nn.ReLU(inplace=True) # the non-linearity
        
        # specify fc layer 2 - in 120, out 84
        self.linear2 = nn.Linear(120, 84, bias=True) # the linearity W*x+b
        self.relu2 = nn.ReLU(inplace=True) # the non-linarity
        
        # specify fc layer 3 - in 84, out 10
        self.linear3 = nn.Linear(84, 10) # the linearity W*x+b
        
        # add a softmax to the last layer
        self.logsoftmax = nn.LogSoftmax(dim=1) # the softmax
        
    # define network forward pass
    def forward(self, images):
        
        # high-level feature learning via convolutional layers
        
        # define conv layer 1 forward pass
        x = self.pool1(self.relu1(self.conv1(images)))
        
        # define conv layer 2 forward pass
        x = self.pool2(self.relu2(self.conv2(x)))
        
        # feature flattening
        
        # reshape image pixels
        x = x.view(-1, 16 * 5 * 5)
        
        # combination of feature learning via non-linear layers
        
        # define fc layer 1 forward pass
        x = self.relu1(self.linear1(x))
        
        # define fc layer 2 forward pass
        x = self.relu2(self.linear2(x))
        
        # define layer 3 forward pass
        x = self.logsoftmax(self.linear3(x))
        
        # return forward pass result
        return x

In [ ]:
model = CIFAR10Net()

In [ ]:
# set cpu or gpu enabled device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu').type

# init deterministic GPU seed
torch.cuda.manual_seed(seed_value)

# log type of device enabled
print('[LOG] notebook with {} computation enabled'.format(str(device)))

[LOG] notebook with cuda computation enabled


In [ ]:
model = model.to(device)

In [ ]:
# define the optimization criterion / loss function
nll_loss = nn.NLLLoss()

In [ ]:
nll_loss = nll_loss.to(device)

In [ ]:
# define learning rate and optimization strategy
learning_rate = 0.001
optimizer = torch.optim.SGD(params=model.parameters(), lr=learning_rate)

In [ ]:
# specify the training parameters
num_epochs = 20 # number of training epochs
mini_batch_size = 128 # size of the mini-batches

In [ ]:
train_dl = DataLoader(train_ds, batch_size=mini_batch_size, shuffle=True, num_workers=2, pin_memory=True)
#train_dl = DeviceDataLoader(train_dl, device)

In [ ]:
transf = torchvision.transforms.Compose([torchvision.transforms.ToTensor(), torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
cifar10_train_data = torchvision.datasets.CIFAR10(root=BASE_PATH, train=True, transform=transf, download=True)
cifar10_train_dataloader = torch.utils.data.DataLoader(cifar10_train_data, batch_size=mini_batch_size, shuffle=True)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting /content/drive/MyDrive/ML/data/cifar-10-python.tar.gz to /content/drive/MyDrive/ML/data


In [ ]:
print(cifar10_train_data[0][0].size())

NameError: ignored

In [ ]:
print(cifar10_train_data[0][0])

In [ ]:
# init collection of training epoch losses
train_epoch_losses = []

# set the model in training mode
model.train()

# train the CIFAR10 model
for epoch in range(num_epochs):
    
    # init collection of mini-batch losses
    train_mini_batch_losses = []

    # iterate over all-mini batches
    for i, (images, labels) in enumerate(train_dl):

        #print(images)

        # push mini-batch data to computation device
        images = images.to(device)
        labels = labels.to(device)

        # run forward pass through the network
        output = model(images)
        
        # reset graph gradients
        model.zero_grad()
        
        # determine classification loss
        loss = nll_loss(output, labels)
        
        # run backward pass
        loss.backward()
        
        # update network paramaters
        optimizer.step()
        
        # collect mini-batch reconstruction loss
        train_mini_batch_losses.append(loss.data.item())

    # determine mean min-batch loss of epoch
    train_epoch_loss = np.mean(train_mini_batch_losses)
    
    # print epoch loss
    now = datetime.utcnow().strftime("%Y%m%d-%H:%M:%S")
    print('[LOG {}] epoch: {} train-loss: {}'.format(str(now), str(epoch), str(train_epoch_loss)))
    
    # set filename of actual model
    model_name = 'challenge_model_epoch_{}.pth'.format(str(epoch))

    # save current model to GDrive models directory
    torch.save(model.state_dict(), os.path.join(MODEL_PATH, model_name))
    
    # determine mean min-batch loss of epoch
    train_epoch_losses.append(train_epoch_loss)

Streaming output truncated to the last 5000 lines.
6
1574
584
1
1
1963
416
6
4
2343
7
338
0
2829
2010
8
2
2210
3
2773
106
4
7
1355
2400
8
6
1782
3
215
1714
9
9
1009
1620
5
2945
8
149
1
7
2362
2
2929
51
0
2
278
7
2543
1623
4
1973
8
1350
0
0
1123
1484
9
7
1227
1062
8
6
2091
2
503
3
584
1467
3
7
1544
8
1609
7
693
761
0
0
2102
1324
4
4
2692
196
0
3
1470
597
5
4
2196
7
2001
1204
2
6
1574
9
2281
6
360
1414
4
5
2453
542
6
2261
6
3
2223
152
2
7
607
6
2581
23
5
721
1
7
2182
334
9
1
653
817
8
8
1265
2106
3
62
8
4
473
4
1038
4
1478
2846
0
9
691
811
0
1
2290
2156
8
2
2963
826
1
7
1813
2
1527
9
1372
0
1492
6
2214
2370
6
1564
3
6
2221
0
2082
759
0
2
1953
6
2057
4
569
87
7
7
1435
1412
7
1901
9
0
1400
970
9
1
1057
4
2095
3
389
9
2215
1138
2
8
2283
122
1
9
2957
0
2276
1802
9
6
1245
7
200
8
1537
5
762
1
682
1616
5
9
1981
7
686
2293
3
1819
2
8
45
1135
9
4
1053
6
1415
7
2201
4
66
7
2366
1076
2
9
1205
0
1487
7
1340
2358
7
8
1564
8
1598
8
728
169
5
3
1413
2349
6
5
2444
2260
1
1821
1
4
1330
369
3
5
378
8
618

Visualize training loss per epoch

In [ ]:
# prepare plot
fig = plt.figure()
ax = fig.add_subplot(111)

# add grid
ax.grid(linestyle='dotted')

# plot the training epochs vs. the epochs' classification error
ax.plot(np.array(range(1, len(train_epoch_losses)+1)), train_epoch_losses, label='epoch loss (blue)')

# add axis legends
ax.set_xlabel("[training epoch $e_i$]", fontsize=10)
ax.set_ylabel("[Classification Error $\mathcal{L}^{NLL}$]", fontsize=10)

# set plot legend
plt.legend(loc="upper right", numpoints=1, fancybox=True)

# add plot title
plt.title('Training Epochs $e_i$ vs. Classification Error $L^{NLL}$', fontsize=10);

Evaluation

In [ ]:
# restore pre-trained model snapshot
best_model_name = os.path.join(models_directory, 'cifar10_model_epoch_19.pth')

# load state_dict from path
state_dict_best = torch.load(best_model_name, map_location=torch.device('cpu'))

# init pre-trained model class
best_model = CIFAR10Net()

# load pre-trained models
best_model.load_state_dict(state_dict_best)

In [ ]:
# set model in evaluation mode
best_model.eval()

In [ ]:
cifar10_eval_dataloader = torch.utils.data.DataLoader(cifar10_eval_data, batch_size=10000, shuffle=False)

In [ ]:
# init collection of mini-batch losses
eval_mini_batch_losses = []

# iterate over all-mini batches
for i, (images, labels) in enumerate(cifar10_eval_dataloader):

    # run forward pass through the network
    output = best_model(images)

    # determine classification loss
    loss = nll_loss(output, labels)

    # collect mini-batch reconstruction loss
    eval_mini_batch_losses.append(loss.data.item())

# determine mean min-batch loss of epoch
eval_loss = np.mean(eval_mini_batch_losses)

# print epoch loss
now = datetime.utcnow().strftime("%Y%m%d-%H:%M:%S")
print('[LOG {}] eval-loss: {}'.format(str(now), str(eval_loss)))

Sample prediction

In [ ]:
# set (random) image id
image_id = 777

# retrieve image exhibiting the image id
cifar10_eval_image, cifar10_eval_label = cifar10_eval_data[image_id]

True class of image

In [ ]:
cifar10_classes[cifar10_eval_label]

Inspect the image

In [ ]:
# define tensor to image transformation
trans = torchvision.transforms.ToPILImage()

# set image plot title 
plt.title('Example: {}, Label: {}'.format(str(image_id), str(cifar10_classes[cifar10_eval_label])))

# un-normalize cifar 10 image sample
cifar10_eval_image_plot = cifar10_eval_image / 2.0 + 0.5

# plot cifar 10 image sample
plt.imshow(trans(cifar10_eval_image_plot))

Compare true label with prediction

In [ ]:
cifar10_eval_image.unsqueeze(0).shape
best_model(cifar10_eval_image.unsqueeze(0))

Obtain all the predictions

In [ ]:
predictions = torch.argmax(best_model(iter(cifar10_eval_dataloader).next()[0]), dim=1)

Overall classification accuracy

In [ ]:
metrics.accuracy_score(cifar10_eval_data.targets, predictions.detach())

Heatmap

In [ ]:
# determine classification matrix of the predicted and target classes
mat = confusion_matrix(cifar10_eval_data.targets, predictions.detach())

# initialize the plot and define size
plt.figure(figsize=(8, 8))

# plot corresponding confusion matrix
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False, cmap='YlOrRd_r', xticklabels=cifar10_classes, yticklabels=cifar10_classes)
plt.tick_params(axis='both', which='major', labelsize=8, labelbottom = False, bottom=False, top = False, left = False, labeltop=True)

# set plot title
plt.title('CIFAR-10 classification matrix')

# set plot axis lables
plt.xlabel('[true label]')
plt.ylabel('[predicted label]');